# Установим MXNet

In [0]:
!pip install mxnet

  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/85/48/85d4295017a67325499c5293dcee334ed40004e5ff0e1d3ffbaac266889a/mxnet_cu80-1.2.0-py2.py3-none-manylinux1_x86_64.whl
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/53/39/4ab213673844e0c004bed8a0781a0721a3f6bb23eb8854ee75c236428892/graphviz-0.8.4-py2.py3-none-any.whl


# Dataflow programming
**Dataflow programming** - гибкий способ задвать параллельные вычисления, где данные "текут" через граф. Граф задаёт порядок операций, то есть - нужно ли их запускать последовательно или параллельно. Каждая операция - **чёрный ящик**: мы только задаём входные и выходные данные, без уточнения их "поведения".

Может звучать как научно-кмопьютерная мумбо-юмба, но эта модель то, что нам нужно для создания нейросети: дать данным проходить определённую последовательность операций, называемых "слои", с каждым слоем, выполняющим инструкции параллельно.

Пример, вот как мы определим Е, как $(A \cdot B) + (C \cdot D)$ ![alt text](https://cdn-images-1.medium.com/max/800/1*h0M4n_9FPyriCwT-LjE0HQ.png)
> Что такое A, B, C и D на данном этапе не важно. Они - **символы**.

Не имеет значение, каков тип входных данных (числа, векторы, матрицы и т.д.), этот граф говорит, как посчитать выходное значение - предоставляя операции "+" и $\cdot$.
> Граф также говорит, что $(A \cdot B) $ и $ (C \cdot D)$ могут быть вычислены **параллельно**.

MXNet будет использовать эту информацию для оптимизации.

# Символы
Теперь мы знаем, почему эти штуки зовутся символами. Взглянем на пример кода.

In [0]:
import mxnet as mx

a = mx.symbol.Variable('A')
b = mx.symbol.Variable('B')
c = mx.symbol.Variable('C')
d = mx.symbol.Variable('D')
e = (a * b) + (c * d)

Мы можем присвоить результат е без знания, чем являются a, b, c и d. Продолжим.

In [0]:
print(a, b, c, d)
print(e)
print(type(e))

<Symbol A> <Symbol B> <Symbol C> <Symbol D>
<Symbol _plus0>
<class 'mxnet.symbol.symbol.Symbol'>


a, b, c и d - символы, как мы явно и указали. е не такой: это тоже символ, но он - результат '+' операции. Посмотрим, что можно ещё узнать.

In [0]:
print(e.list_arguments())
print(e.list_outputs())
print(e.get_internals().list_outputs())

['A', 'B', 'C', 'D']
['_plus0_output']
['A', 'B', '_mul0_output', 'C', 'D', '_mul1_output', '_plus0_output']


Что это нам говорит:
* е зависит от переменных a, b, c  и d;
* операция, из которой получается e,- сложение
* e - это ни что иное, как $(a \cdot b) + (c \cdot d)$

Теперь попробуем применить это на практике.

# Связывание NDArrays и Символов
> Применение шагов для вычислений определённых через символы, к данным, хранящимся в NDArrays, требует операции под названием "привязка" (binding).
Продолжим с примером сверху. Здесь я присвою 'A' - 1, 'B' - 2, 'C' - 3, 'D' - 4, вот почему я создаю 4 массива, содержащих одно целое число.


In [0]:
import numpy as np

In [0]:
a_data = mx.nd.array([1], dtype=np.int32)
b_data = mx.nd.array([2], dtype=np.int32)
c_data = mx.nd.array([3], dtype=np.int32)
d_data = mx.nd.array([4], dtype=np.int32)

Дальше свяжем каждый массив с соотвествующим Символом. Заметьте, что мне нужно выбрать **context** (CPU или GPU), где будут происходить вычисления.

In [0]:
executor = e.bind(ctx=mx.cpu(), args={'A': a_data, 'B': b_data, 'C': c_data, 
                                      'D': d_data})
print(executor)

Теперь пришло время прогнать входные данные через граф, чтобы получить результат: функция ```forward()``` с этим поможет. Она возвращает массив NDArrays, т.к. граф может иметь множество результатов. Здесь же у нас единственный вывод, содержащий число 14, что как раз есть $(1 \cdot 2) + (3 \cdot 4)$.

In [0]:
e_data = executor.forward()
print(e_data)
print(e_data[0])

[
[14]
<NDArray 1 @cpu(0)>]

[14]
<NDArray 1 @cpu(0)>


Попробуем применить тот же самый граф к четырём матрицам 1000х1000, заполненных рандомными числами от 0 до 1. Всё, что нам нужно сделать, так это определить новые входные данные: связывание и вычисления те же самые.

In [0]:
a_data = mx.nd.uniform(low=0, high=1, shape=(1000, 1000))
b_data = mx.nd.uniform(low=0, high=1, shape=(1000, 1000))
c_data = mx.nd.uniform(low=0, high=1, shape=(1000, 1000))
d_data = mx.nd.uniform(low=0, high=1, shape=(1000, 1000))

executor = e.bind(ctx=mx.cpu(), args={'A': a_data, 'B': b_data, 'C': c_data, 
                                      'D': d_data})
e_data = executor.forward()

In [0]:
print(e_data)

[
[[0.1977522  0.58930916 0.9957535  ... 0.28916416 0.3330607  0.5123714 ]
 [0.07656978 0.5997503  0.55926484 ... 0.38490385 0.36591023 0.45478362]
 [0.39658463 0.17304057 0.54193836 ... 0.816297   0.7078872  0.52949476]
 ...
 [0.28478235 0.57425654 0.5903597  ... 0.2610643  0.82782537 0.73320323]
 [0.42121097 0.6792569  0.82689947 ... 0.06357475 0.21329054 0.6004921 ]
 [0.84038836 0.43226227 0.25343633 ... 0.9770783  0.5826757  0.32416496]]
<NDArray 1000x1000 @cpu(0)>]


In [0]:
print(e_data[0])


[[0.1977522  0.58930916 0.9957535  ... 0.28916416 0.3330607  0.5123714 ]
 [0.07656978 0.5997503  0.55926484 ... 0.38490385 0.36591023 0.45478362]
 [0.39658463 0.17304057 0.54193836 ... 0.816297   0.7078872  0.52949476]
 ...
 [0.28478235 0.57425654 0.5903597  ... 0.2610643  0.82782537 0.73320323]
 [0.42121097 0.6792569  0.82689947 ... 0.06357475 0.21329054 0.6004921 ]
 [0.84038836 0.43226227 0.25343633 ... 0.9770783  0.5826757  0.32416496]]
<NDArray 1000x1000 @cpu(0)>


In [0]:
print(e_data[0].asnumpy())

[[0.1977522  0.58930916 0.9957535  ... 0.28916416 0.3330607  0.5123714 ]
 [0.07656978 0.5997503  0.55926484 ... 0.38490385 0.36591023 0.45478362]
 [0.39658463 0.17304057 0.54193836 ... 0.816297   0.7078872  0.52949476]
 ...
 [0.28478235 0.57425654 0.5903597  ... 0.2610643  0.82782537 0.73320323]
 [0.42121097 0.6792569  0.82689947 ... 0.06357475 0.21329054 0.6004921 ]
 [0.84038836 0.43226227 0.25343633 ... 0.9770783  0.5826757  0.32416496]]
